**Loading TimesFM Model**

In [17]:
import os
import glob
import time
from datetime import datetime
import pandas as pd
import numpy as np
from collections import defaultdict

import timesfm

In [18]:
tfm = timesfm.TimesFm(
    context_len=512 ,
    horizon_len=24,
    input_patch_len=32,
    output_patch_len=128,
    num_layers=20,
    model_dims=1280,
    backend='cpu'
)
tfm.load_from_checkpoint(repo_id="google/timesfm-1.0-200m")

Multiprocessing context has already been set.


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Constructing model weights.


Constructed model weights in 4.82 seconds.
Restoring checkpoint from /home/user/.cache/huggingface/hub/models--google--timesfm-1.0-200m/snapshots/8775f7531211ac864b739fe776b0b255c277e2be/checkpoints.


ERROR:absl:For checkpoint version > 1.0, we require users to provide
          `train_state_unpadded_shape_dtype_struct` during checkpoint
          saving/restoring, to avoid potential silent bugs when loading
          checkpoints to incompatible unpadded shapes of TrainState.


Restored checkpoint in 0.52 seconds.
Jitting decoding.
Jitted decoding in 28.41 seconds.


In [19]:
# Data pipelining
def get_batched_data_fn(sub_df,
    batch_size: int = 128, 
    context_len: int = 168, 
    horizon_len: int = 24):
    
    examples = defaultdict(list)
    num_examples = 0
    for start in range(0, len(sub_df) - (context_len + horizon_len), horizon_len):
      num_examples += 1
      examples["inputs"].append(sub_df["y"][start:(context_end := start + context_len)].tolist())
      examples["outputs"].append(sub_df["y"][context_end:(context_end + horizon_len)].tolist())
      examples['inputs_ts'].append(sub_df["ds"][start:(context_end := start + context_len)].tolist())
      examples["outputs_ts"].append(sub_df["ds"][context_end:(context_end + horizon_len)].tolist())

  
    def data_fn():
        for i in range(1 + (num_examples - 1) // batch_size):
            yield {k: v[(i * batch_size) : ((i + 1) * batch_size)] for k, v in examples.items()}
  
    return data_fn

In [20]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale

# Benchmark
batch_size = 32
context_len = 168
horizon_len = 24

def process_building(df):
 
    input_data = get_batched_data_fn(df, batch_size=500)

    metrics = defaultdict(list)
    results_all = []
    for i, example in enumerate(input_data()):
    
        raw_forecast, _ = tfm.forecast(inputs=example["inputs"], freq=[0] * len(example["inputs"]))

        for ts, y_true, y_pred in zip(example['outputs_ts'], example['outputs'], raw_forecast):
            res_df = pd.DataFrame({'ts': ts, 'y_true': y_true,'y_pred': y_pred})
            results_all.append(res_df)
        
    results_all_df = pd.concat(results_all)
    return results_all_df

def process_file(filename):
    df = pd.read_csv(filename)
    df = df.set_index(['timestamp'])

    if df.shape[1] < 2:
        return None
        
    print(datetime.now(), df.shape, flush=True)

    results_all = []
    i =0
    for building_name in df.columns:
        print(datetime.now(), building_name, flush=True)
        df1 = df[[building_name]]
        df1 = df1.reset_index()
        df1.columns = ['ds', 'y']

        df1['y'] = minmax_scale(df1['y'])

        res = process_building(df1)
        res['building'] = building_name
        results_all.append(res)
        i+=1

        
    results_all_df = pd.concat(results_all)
    return results_all_df

In [21]:
files_list = glob.glob('/home/user/BuildingsBenchNREL/BuildingsBenchData/BuildingsBench/Buildings-900K-test-csv/*.csv')  #Enter Dataset location in CSV format

dataset = 'Buildings-900k-test'  #Dataset name to be processed
os.makedirs(f'./forecasts/{dataset}/', exist_ok = True)
os.makedirs(f'./results/{dataset}/', exist_ok = True)

for filename in files_list:
    print(datetime.now(), filename)
    results = process_file(filename)
    if results is not None:
        results.to_csv(f'./forecasts/{dataset}/{os.path.basename(filename)}')
    print('')

2024-07-20 04:26:26.133980 /home/user/BuildingsBenchNREL/BuildingsBenchData/BuildingsBench/Buildings-900K-test-csv/G51059305.csv
2024-07-20 04:26:26.297669 (8759, 83)
2024-07-20 04:26:26.298689 4390
2024-07-20 04:26:29.562737 16464
2024-07-20 04:26:32.900243 19360
2024-07-20 04:26:36.254478 23149
2024-07-20 04:26:39.459494 23184
2024-07-20 04:26:42.796585 27072
2024-07-20 04:26:46.363593 32240
2024-07-20 04:26:49.700873 33667
2024-07-20 04:26:53.048002 41024
2024-07-20 04:26:56.261165 41181
2024-07-20 04:26:59.576245 44720
2024-07-20 04:27:02.935932 45014
2024-07-20 04:27:06.277489 46221
2024-07-20 04:27:09.641037 60979
2024-07-20 04:27:12.988319 61645
2024-07-20 04:27:16.203401 65315
2024-07-20 04:27:19.540081 65731
2024-07-20 04:27:22.735931 66947
2024-07-20 04:27:26.405929 69584
2024-07-20 04:27:29.574589 69942
2024-07-20 04:27:32.910435 85712
2024-07-20 04:27:36.238554 92493
2024-07-20 04:27:39.623515 93197
2024-07-20 04:27:42.846633 94963
2024-07-20 04:27:46.235474 95088
2024-07-2

**Metrics**

In [18]:
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import root_mean_squared_log_error
from permetrics.regression import RegressionMetric

dataset = 'Buildings-900k-test'
files_list = glob.glob(f'./forecasts/{dataset}/*.csv')

metrics_all_files = []

for filename in files_list:
    res = pd.read_csv(filename)
    metrics_all = []
    for (g, data) in res.groupby(['building']):
        data = data.dropna()
        data = data[data.y_pred >= 0]
        print(g)  
        rmse= root_mean_squared_error(data.y_true, data.y_pred)
        mae= mean_absolute_error(data.y_true, data.y_pred)
        mape = mean_absolute_percentage_error(data.y_true, data.y_pred)
        mse= mean_squared_error(data.y_true, data.y_pred)
        msle= mean_squared_log_error(data.y_true, data.y_pred)
        rmsle= root_mean_squared_log_error(data.y_true, data.y_pred)
        nrmse = rmse / (data.y_true.mean()) 
    
        evaluator = RegressionMetric(data.y_true.to_list(), data.y_pred.to_list())
        smape= evaluator.symmetric_mean_absolute_percentage_error()
    
        metrics = pd.DataFrame({'building_name': [g[0]], 
                           'mae': [mae],
                            'mape': [mape],
                           'mse': [mse], 'rmse': [rmse], 'msle': [msle], 'rmsle': [rmsle], 'nrmse' : [nrmse],
                               'sMAPE' : [smape]})
        metrics_all.append(metrics)
    
    metrics_all_df = pd.concat(metrics_all)
    metrics_all_df.to_csv(f'./results/{dataset}/{os.path.basename(filename)}')

    metrics_all_df['filename'] = os.path.basename(filename)
    metrics_all_files.append(metrics_all_df)

metrics_all_files_df = pd.concat(metrics_all_files)

(4390,)
(16464,)
(19360,)
(23149,)
(23184,)
(27072,)
(32240,)
(33667,)
(41024,)
(41181,)
(44720,)
(45014,)
(46221,)
(60979,)
(61645,)
(65315,)
(65731,)
(66947,)
(69584,)
(69942,)
(85712,)
(92493,)
(93197,)
(94963,)
(95088,)
(99053,)
(100419,)
(104032,)
(109155,)
(116304,)
(122131,)
(124045,)
(126813,)
(129651,)


/home/user/miniconda3/envs/tfm_env/lib/python3.10/site-packages/permetrics/regression.py:357: RuntimeWarning: invalid value encountered in divide
  result = np.mean(np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)), axis=0)
/home/user/miniconda3/envs/tfm_env/lib/python3.10/site-packages/permetrics/regression.py:357: RuntimeWarning: invalid value encountered in divide
  result = np.mean(np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)), axis=0)
/home/user/miniconda3/envs/tfm_env/lib/python3.10/site-packages/permetrics/regression.py:357: RuntimeWarning: invalid value encountered in divide
  result = np.mean(np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)), axis=0)
/home/user/miniconda3/envs/tfm_env/lib/python3.10/site-packages/permetrics/regression.py:357: RuntimeWarning: invalid value encountered in divide
  result = np.mean(np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)), axis=0)
/home/user/miniconda3/envs/tfm_env/lib/python3.10/site-packages/

(131315,)
(139830,)
(141203,)
(144166,)
(147955,)
(150464,)
(156470,)
(160294,)
(162294,)
(164573,)
(166003,)
(167382,)
(168752,)
(179110,)
(182643,)
(186253,)
(188704,)
(189891,)
(212192,)
(215885,)
(216589,)
(216816,)
(216947,)
(217110,)
(221792,)
(245267,)
(247437,)
(250589,)
(251776,)
(257539,)
(260691,)
(275363,)
(285392,)
(289885,)


/home/user/miniconda3/envs/tfm_env/lib/python3.10/site-packages/permetrics/regression.py:357: RuntimeWarning: invalid value encountered in divide
  result = np.mean(np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)), axis=0)
/home/user/miniconda3/envs/tfm_env/lib/python3.10/site-packages/permetrics/regression.py:357: RuntimeWarning: invalid value encountered in divide
  result = np.mean(np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)), axis=0)
/home/user/miniconda3/envs/tfm_env/lib/python3.10/site-packages/permetrics/regression.py:357: RuntimeWarning: invalid value encountered in divide
  result = np.mean(np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)), axis=0)
/home/user/miniconda3/envs/tfm_env/lib/python3.10/site-packages/permetrics/regression.py:357: RuntimeWarning: invalid value encountered in divide
  result = np.mean(np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)), axis=0)
/home/user/miniconda3/envs/tfm_env/lib/python3.10/site-packages/

(293104,)
(297206,)
(298192,)
(299856,)
(301315,)
(307568,)
(307597,)
(312160,)
(313846,)
(337536,)
(337933,)
(338160,)
(343232,)
(345283,)
(348278,)


...

/home/user/miniconda3/envs/tfm_env/lib/python3.10/site-packages/permetrics/regression.py:357: RuntimeWarning: invalid value encountered in divide
  result = np.mean(np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)), axis=0)
/home/user/miniconda3/envs/tfm_env/lib/python3.10/site-packages/permetrics/regression.py:357: RuntimeWarning: invalid value encountered in divide
  result = np.mean(np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)), axis=0)
/home/user/miniconda3/envs/tfm_env/lib/python3.10/site-packages/permetrics/regression.py:357: RuntimeWarning: invalid value encountered in divide
  result = np.mean(np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)), axis=0)
/home/user/miniconda3/envs/tfm_env/lib/python3.10/site-packages/permetrics/regression.py:357: RuntimeWarning: invalid value encountered in divide
  result = np.mean(np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)), axis=0)
/home/user/miniconda3/envs/tfm_env/lib/python3.10/site-packages/

(332524,)
(332652,)
(332940,)
(335159,)
(345964,)
(346295,)
(348364,)
(7022,)
(7726,)
(17902,)
(20517,)
(21373,)
(23150,)
(30430,)
(32014,)
(39333,)
(41862,)
(43669,)
(45518,)
(55214,)
(59334,)
(61262,)
(65573,)
(77070,)
(80222,)
(91270,)
(95950,)
(96085,)
(100270,)
(106702,)
(113166,)
(117501,)
(120189,)
(122357,)
(124046,)
(125958,)
(126814,)
(127198,)
(133726,)
(141173,)
(143357,)


/home/user/miniconda3/envs/tfm_env/lib/python3.10/site-packages/permetrics/regression.py:357: RuntimeWarning: invalid value encountered in divide
  result = np.mean(np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)), axis=0)
/home/user/miniconda3/envs/tfm_env/lib/python3.10/site-packages/permetrics/regression.py:357: RuntimeWarning: invalid value encountered in divide
  result = np.mean(np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)), axis=0)
/home/user/miniconda3/envs/tfm_env/lib/python3.10/site-packages/permetrics/regression.py:357: RuntimeWarning: invalid value encountered in divide
  result = np.mean(np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)), axis=0)


(144997,)
(145774,)
(152798,)
(169749,)
(176622,)
(177461,)
(183509,)
(188166,)
(209582,)
(212733,)
(215886,)
(216590,)
(217109,)
(218453,)
(230094,)
(235166,)
(245493,)
(256253,)
(259965,)
(260789,)
(261742,)
(274822,)
(277973,)
(288654,)
(292878,)
(293269,)
(295037,)
(297326,)
(303101,)
(313845,)
(316078,)
(316213,)
(319902,)
(323861,)
(325070,)
(331630,)
(341525,)
(349358,)


/home/user/miniconda3/envs/tfm_env/lib/python3.10/site-packages/permetrics/regression.py:357: RuntimeWarning: invalid value encountered in divide
  result = np.mean(np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)), axis=0)
/home/user/miniconda3/envs/tfm_env/lib/python3.10/site-packages/permetrics/regression.py:357: RuntimeWarning: invalid value encountered in divide
  result = np.mean(np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)), axis=0)
/home/user/miniconda3/envs/tfm_env/lib/python3.10/site-packages/permetrics/regression.py:357: RuntimeWarning: invalid value encountered in divide
  result = np.mean(np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)), axis=0)
/home/user/miniconda3/envs/tfm_env/lib/python3.10/site-packages/permetrics/regression.py:357: RuntimeWarning: invalid value encountered in divide
  result = np.mean(np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)), axis=0)
/home/user/miniconda3/envs/tfm_env/lib/python3.10/site-packages/

In [19]:
metrics_all_files_df.to_csv(f'./results/{dataset}/results_combined.csv')
metrics_all_files_df

,building_name,mae,mape,mse,rmse,msle,rmsle,nrmse,nrmse_eve,sMAPE,filename
0,4390,0.044350,1.982268e+10,0.005398,0.073473,0.003617,0.060140,0.519121,0.819596,0.184059,G51059305.csv
0,16464,0.047075,2.039306e+13,0.006058,0.077835,0.003906,0.062500,0.440327,0.703820,1.000000,G51059305.csv
0,19360,0.052865,3.825223e+10,0.007018,0.083775,0.004442,0.066645,0.500203,0.619901,0.177993,G51059305.csv
0,23149,0.047253,2.452815e+10,0.004594,0.067781,0.003062,0.055332,0.353350,0.626946,0.143902,G51059305.csv
0,23184,0.046728,3.607117e+10,0.004906,0.070046,0.003143,0.056066,0.360782,0.618265,0.156685,G51059305.csv
...,...,...,...,...,...,...,...,...,...,...,...
0,323861,0.040931,1.249692e+10,0.002988,0.054667,0.001407,0.037506,0.106005,0.225848,0.081558,G06000702.csv
0,325070,0.035951,5.020852e+10,0.002280,0.047747,0.001131,0.033628,0.108167,0.188498,0.083021,G06000702.csv
0,331630,0.052720,4.902214e+10,0.004963,0.070446,0.002362,0.048604,0.148224,0.336855,0.087934,G06000702.csv
0,341525,0.125680,1.346135e+12,0.027491,0.165805,0.015714,0.125357,0.587368,1.003440,0.254185,G06000702.csv


In [20]:
metrics_all_files_df.describe()*100

,building_name,mae,mape,mse,rmse,msle,rmsle,nrmse,nrmse_eve,sMAPE
count,5.650000e+04,56500.000000,5.650000e+04,56500.000000,56500.000000,56500.000000,56500.000000,56500.000000,56500.000000,56500.000000
mean,1.739531e+07,4.775962,2.551663e+14,0.588556,7.410366,0.336306,5.613102,34.394274,49.373834,34.145142
std,1.008230e+07,1.430760,1.144320e+15,0.413374,1.987219,0.220794,1.458576,12.162235,14.634890,36.751169
min,9.000000e+04,2.346076,1.197518e+01,0.136609,3.696060,0.063485,2.519619,7.640469,13.481807,5.955783
25%,8.971600e+06,4.008459,1.859588e+12,0.393198,6.270552,0.225003,4.743451,25.184696,39.172238,12.099454
50%,1.697490e+07,4.564657,5.170670e+12,0.501011,7.078213,0.296284,5.443195,33.467723,49.081178,15.255639
75%,2.611880e+07,5.164689,3.007759e+13,0.643350,8.020911,0.374748,6.121664,42.802448,58.790682,23.977088
max,3.498280e+07,15.905435,2.363292e+16,5.874269,24.236891,3.039627,17.434527,76.949468,116.783078,100.000000
